In [1]:
import scipy
import numpy as np
from scipy.optimize import minimize_scalar, minimize, LinearConstraint

- minimize_scalar() and minimize() to minimize a function of one ariable and many variables, respectively
- curve_fit() to fit a function to a set of data
- root_scalar() and root() to find the zeros of a function of one - variable and many variables, respectively
- linprog() to minimize a linear objective function with linear inequality and equality constraints

## Minimizing a Function With One Variable

Note: As you may know, not every function has a minimum. For instance, try and see what happens if your objective function is y = x³. For minimize_scalar(), objective functions with no minimum often result in an OverflowError because the optimizer eventually tries a number that is too big to be calculated by the computer.

On the opposite side of functions with no minimum are functions that have several minima. In these cases, minimize_scalar() is **not guaranteed to find the global minimum of the function**. However, minimize_scalar() has a method keyword argument that you can specify to control the solver that’s used for the optimization. The SciPy library has three built-in methods for scalar minimization:

1. brent is an implementation of Brent’s algorithm. This method is the default.
2. golden is an implementation of the golden-section search. The documentation notes that Brent’s method is usually better.
3. bounded is a bounded implementation of Brent’s algorithm. It’s useful to limit the search region when the minimum is in a known range.

When method is either brent or golden, minimize_scalar() takes another argument called bracket. This is a sequence of two or three elements that provide an initial guess for the bounds of the region with the minimum. **However, these solvers do not guarantee that the minimum found will be within this range.**

On the other hand, when method is bounded, minimize_scalar() takes another argument called bounds. This is a sequence of two elements that strictly bound the search region for the minimum. Try out the bounded method with the function y = x⁴ - x². This function is plotted in the figure below:

In [2]:
# a simple example, only 1 minima
def objectiv_function(x):
    return 3 * x ** 4 - 2 * x + 1

res = minimize_scalar(objectiv_function)

res

     fun: 0.17451818777634331
    nfev: 16
     nit: 12
 success: True
       x: 0.5503212087491959

In [3]:
# use `bounded` method to search minima within some interval
def objective_function(x):
    return x ** 4 - x ** 2

res = minimize_scalar(
    objective_function,
    method='bounded',
    bounds=(-1, 0),
)

res

     fun: -0.24999999999998732
 message: 'Solution found.'
    nfev: 10
  status: 0
 success: True
       x: -0.707106701474177

## Minimizing a Function With Many Variables

ref: [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)

In [4]:
# generate data
n_buyers = 10
n_shares = 15

np.random.seed(10)
prices = np.random.random(n_buyers)
money_available = np.random.randint(1, 4, n_buyers)
n_shares_per_buyer = money_available / prices

print(prices, money_available, n_shares_per_buyer, sep="\n")

[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701 0.22479665
 0.19806286 0.76053071 0.16911084 0.08833981]
[1 1 1 3 1 3 3 2 1 1]
[ 1.29647768 48.18824404  1.57816269  4.00638948  2.00598984 13.34539487
 15.14670609  2.62974258  5.91328161 11.3199242 ]


In [5]:
# NOTE: constraint must be a list of dicts
constraint = [{'type': 'eq', 'fun': lambda x: sum(x) - n_shares}]

# create the bounds for the solution variable
bounds = [(0, n) for n in n_shares_per_buyer]

In [6]:
# maximize y -> minimize -y
def objective_function(x, prices):
    return -x.dot(prices)

# optimize with inital guess
res = minimize(
    objective_function,
    x0=10 * np.random.random(n_buyers), # inital guess
    args=(prices,), # additional arguments to objective_function()
    constraints=constraint,
    bounds=bounds,
)

res

     fun: -8.783020157087478
     jac: array([-0.7713207 , -0.02075195, -0.63364828, -0.74880385, -0.49850702,
       -0.22479665, -0.1980629 , -0.76053071, -0.16911089, -0.08833981])
 message: 'Optimization terminated successfully.'
    nfev: 204
     nit: 17
    njev: 17
  status: 0
 success: True
       x: array([1.29647768e+00, 2.78286565e-13, 1.57816269e+00, 4.00638948e+00,
       2.00598984e+00, 3.48323773e+00, 3.19744231e-14, 2.62974258e+00,
       2.38121197e-14, 8.84962214e-14])

In [7]:
print(f"The total number of shares is: {sum(res.x)}")
print(f"Leftover money for each buyer: {money_available - res.x * prices}")

The total number of shares is: 15.000000000000002
Leftover money for each buyer: [3.08642001e-14 1.00000000e+00 3.09752224e-14 6.48370246e-14
 3.28626015e-14 2.21697984e+00 3.00000000e+00 6.46149800e-14
 1.00000000e+00 1.00000000e+00]
